### OCI Data Science - Useful Tips
Everything stored in the <span style="background-color: #d5d8dc ">/home/datascience</span> folder is now stored on your block volume drive. The <span style="background-color: #d5d8dc ">ads-examples</span> folder has moved outside of your working space. Notebook examples are now accessible through a Launcher tab "Notebook Examples" button.
<details>
<summary><font size="2">1. Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">2. OCI Configuration and Key Files Set Up</font></summary><p>Follow the instructions in the getting-started notebook. That notebook is accessible via the "Getting Started" Launcher tab button.</p>
</details>
<details>
<summary><font size="2">3. Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">4. Typical Cell Imports and Settings</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import MLData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">5. Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import logging
import os
import cx_Oracle
from datetime import datetime
from datetime import timedelta
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

In [2]:
# Add TNS_ADMIN to the environment
os.environ['TNS_ADMIN'] = "/home/datascience/ADB"
connection = cx_Oracle.connect('hackathon', 'bCfrCC0981267', 'adw_low')

TABELA PARA TIPO DE ESTABELECIMENTO

In [97]:
query = pd.read_sql('SELECT   to_number(substr(ICUP_DT_REF,1,6)) as mes, MERC_DS_TIPO_ESTABELECIMENTO as tipo, count(distinct CPF_CRIP) as QTD_CLIENTES, count( CPF_CRIP) as QTD_TRANSACOES, round(sum(ICUP_VL_TOTALITEM),2) AS VLR_TOTAL ,sum(ICUP_VL_PAGOCARTAO) AS VLR_PAGO_CARTAO, round(sum(ICUP_VL_TOTALITEM)/count(distinct CPF_CRIP),2) TICKET_MEDIO_CLIENTE from EVE_BUFFER_CCI_AQUISICAO_HCKT  where to_number(substr(ICUP_DT_REF,1,6)) >= 202001 group by MERC_DS_TIPO_ESTABELECIMENTO,to_number(substr(ICUP_DT_REF,1,6)) order by 1, 2', con=connection) 
query.to_csv (r'export_dataframe.csv', index = False, header=True)
query

,MES,TIPO,QTD_CLIENTES,QTD_TRANSACOES,VLR_TOTAL,VLR_PAGO_CARTAO,TICKET_MEDIO_CLIENTE
0,202001,10-E-COMMERCE,218,4825,79436.56,15328.83,364.39
1,202001,11-MARKET,1144,13862,103560.23,4424.19,90.52
2,202001,12-OUTLET,145,367,56246.46,21206.26,387.91
3,202001,15-MARCA PROPRIA,77,920,5329.38,2695.64,69.21
4,202001,2-DROGARIA,5623,13114,367560.16,113963.20,65.37
5,202001,3-EXPRESS,6387,42392,284074.27,21473.84,44.48
6,202001,4-HIPER,140383,2703236,43420065.51,14228933.84,309.30
7,202001,5-POSTO,26069,47201,4156446.34,1287056.68,159.44
8,202001,7-BAIRRO,10617,192975,1879666.60,259245.18,177.04
9,202001,None,4151,7063,3711766.29,995541.99,894.19


TABELA PARA TRANSPORTE

In [98]:
tabela_2 = pd.read_sql('''
SELECT
--SUBSTR(TAUT_DT_REFAUT,1,6) AS PERIODO
--,
TAUT_DS_CATEGORIA
,COUNT(DISTINCT NUMBERX_CRIPT) AS VOL_CONTAS
,COUNT(NUMBERX_CRIPT) AS VOL_TRANSACOES
,SUM(ROUND(TO_NUMBER(TAUT_VL_AUTORIZACAO),2)) VALOR
,SUM(TAUT_VL_AUTORIZACAO)/COUNT(DISTINCT NUMBERX_CRIPT) AS VL_MEDIO

FROM (SELECT  *
from EVE_AUTHORIZATIONS_MANU_HCKT
where
--to_number(to_char(TAUT_DH_TRANSACAO,'yyyymm')) = 202006 
--AND 
TAUT_ST_TRANSACAO LIKE '%APROVADA%'
AND TAUT_CD_EMPRESA = 1
and 
 (upper(TAUT_DS_ESTABELECIMENTO) like '%UBER TRIP%'
 or
 upper(TAUT_DS_ESTABELECIMENTO) like '%UBER *TRIP%'
   or
 upper(TAUT_DS_ESTABELECIMENTO) like '%PAYPAL *UBER BR%'
    or
 upper(TAUT_DS_ESTABELECIMENTO) like '%PAYPAL *UBER *%'
    or
 upper(TAUT_DS_ESTABELECIMENTO) like '%*UBER99PO%'
     or
 upper(TAUT_DS_ESTABELECIMENTO) like '%PICPAY*JW4*UBERPR%'
 or
upper(TAUT_DS_ESTABELECIMENTO) like '%99*%'
or 
upper(TAUT_DS_ESTABELECIMENTO) like '%CABIFY*%' )
)
where TAUT_DS_CATEGORIA != 'ON-US'
GROUP BY
--SUBSTR(TAUT_DT_REFAUT,1,6)
--,
TAUT_DS_CATEGORIA
order by 1 asc
''', con=connection)
tabela_2 

,TAUT_DS_CATEGORIA,VOL_CONTAS,VOL_TRANSACOES,VALOR,VL_MEDIO
0,OFF-US,147543,2694001,34290225.34,232.408351
